In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import odeint
import pysindy as ps
import math

Variables globales:

In [12]:
paso_tiempo=0.001
max_time = 40.0
coefs = {'A': 5,
               'B': 10,
               'C': 15,
               'D': 2,
               'E': 6,
               'F': 3,
               'G': 1,
               'H': 9,
               'I': 8,
               'J': 7}
inicial_values = {'theta': 5,
                 'alfa': 7,
                 'vel': 4,
                 'theta_dot': 3,
                 'alfa_dot': 2}

Las incógnitas son theta, alfa, V, theta prima y alfa prima. Ecuaciones a modelar:

In [14]:
def eqs(a=coefs['A'], b=coefs['B'], c=coefs['C'], d=coefs['D'], e=coefs['E'], f=coefs['F'], g=coefs['G'],
           h=coefs['H'], i=coefs['I'], j=coefs['J'], theta0=inicial_values['theta'], alfa0=inicial_values['alfa'],
           vel0=inicial_values['vel'], theta_dot0=inicial_values['theta_dot'], alfa_dot0=inicial_values['alfa_dot']):
    
    vel_dot = a*math.cos(alfa0) - b*v**2 - c*math.sin(theta0-alfa0)
    alfa_dot = d*theta_dot0 - e*v*alfa0 - f*math.sin(alfa0)*1/v + g*math.cos(theta0 - alfa0)*1/v
    theta_dot_dot = h*alfa0*v**2 + i*v*theta_dot0 + j*v*alfa_dot0
    
    x_dot = v*math.cos(theta0-alfa0)
    y_dot = v*math.sin(theta0-alfa0)
    return vel_dot, alfa_dot, theta_dot_dot, x_dot, y_dot

In [18]:
def calculate_data(t_max =max_time, dt =paso_tiempo, a=coefs['A'], b=coefs['B'], c=coefs['C'], d=coefs['D'], e=coefs['E'], f=coefs['F'], g=coefs['G'],
           h=coefs['H'], i=coefs['I'], j=coefs['J'], theta0=inicial_values['theta'], alfa0=inicial_values['alfa'],
           vel0=inicial_values['vel'], theta_dot0=inicial_values['theta_dot'], alfa_dot0=inicial_values['alfa_dot']):
    
    t_train = np.arange(0.0, t_max, dt)
          
    #odeint resuelve edos dado en el primer argumento la funcion que devuelve las ecuaciones dx/dt = ...
    x_train = odeint(eqs, x0_train, t_train, args=(a,b,c,d,e,f,g,h,i,j)) 

    x_dot_train_measured = np.ones((t_train.size, 5))
    for i in range(t_train.size):
        x_dot_train_measured[i] = eqs(x_train[i], t_train, a,b,c,d,f,g,h,i,j)


    fig = plt.figure()
    ax = fig.gca(projection='2d')

    ax.plot(x_train[:, 0], x_train[:, 1], lw=0.5)
    ax.set_xlabel("X Axis")
    ax.set_ylabel("Y Axis")
    ax.set_title("Trayectoria")

    plt.show()
    
    return x_train, x_dot_train_measured

In [4]:
def sindy_model(x_train, x_dot_train, poly_order = 4, threshold = 0.2, terminos_ctes=False, dt=paso_tiempo):
    # Fit the model, me creo el modelo dinámico 

     #orden 3 y 4 es el que mejor lo hace
    

    #pysindy calcula las ecs que gobierna el movimiento
    model = ps.SINDy(
        optimizer=ps.STLSQ(threshold=threshold, normalize=True,  fit_intercept=terminos_ctes),
        feature_library=ps.PolynomialLibrary(degree=poly_order, ),#interaction_only=terminos_ctes,
                                                         
        feature_names=['x', 'y', 'z'],
    )

    #la doc dice que en t hay que poner el time step
    model.fit(x_train, t=dt, x_dot=x_dot_train, quiet=True)

    #print(model.equations(precision=52))
    model.print(precision=4)
    #print(model.coefficients())
    return model

Genero los datos:

In [ ]:
x_train, x_dot_train = calculate_data()
model = sindy_model(x_train, x_dot_train)
